In [1]:
import numpy as np
import itertools
from random import randint
from IPython.display import clear_output
from scipy.stats import entropy
from scipy.special import comb
from math import log, factorial, floor, ceil, isinf, isnan

from LPFullBib import CenteredBinomialFullBib
from LPHalfBib import CenteredBinomialHalfBib
from low_perm_bib import low_perm_bib
from Frodo_Stuff import Frodo_Full, Frodo_Dist, Frodo_Bib
from May_Results import Maydist, our_maybib
from Uniformbibs import uniformoptibib
from Deltabib import deltabib

q=3329

# distributions are denoted with arrays of the form [w_0,w_1,...,w_eta].
# distributions need to be of length at least 5, filled up w/ zeroes if necessary.

# parameter sets are stored as dictionaries (referred to as bibs). For example, To call upon
# level 3 parameter epsilon_20^{(3)}, you call Bib_name(20, 3).
# If Bib_name is a Rep-2 parameter set, Bib(30, j), Bib(31, j), Bib(32, j), Bib(33, j)
# exist but are constantly set to 0.0 .

# Each set of parameters can be called upon by utilizing their respective syntax.
# The respective syntaxes differ, depending on the distribution, and can be seen below:

# figures 21, 22, 25, 26, 29: CenteredBinomialFullBib(eta, level)
# figure 23: Frodo_Bib(eta, level) (eta is the maximal value per distribution, i.e. for
#            Frodo-640: eta=12,   Frodo-976: eta=10,   Frodo-1344: eta=6)
# figures 24, 30: CenteredBinomialHalfBib(eta, level)
# figure 27: low_perm_bib(2, eta, level)
# figure 28: low_perm_bib(1, eta, level)
# figure 31: uniformoptibib(level)
# figure 32: deltabib(delta, level)
# figures 33, 34: our_maybib(omega, level)

# To evaluate a certain distribution "dist" with parameter set "bib" for level "j", call
# for runtimes: Evaluate_T(dist, j, bib)
# for memory size: Evaluate_L(dist, j, bib)
# the returned array gives you the calculated values PER LEVEL, i.e. 
# Evaluate_T(dist, j, bib)[0] returns the calculated runtime on level 0

In [2]:
def peta(eta,Permaway=[]):
    # Probability for an randomly drawn entry to be in the permaway-set
    sum=0
    for i in Permaway:
        if i==0:
            sum=sum+comb(2*eta,eta,exact=True)
        else:
            sum=sum+2*comb(2*eta,eta+i,exact=True)
    return sum/(2**(2*eta))

def pbin(eta,i):
    # centered binomial distribution probabilities
    return comb(2*eta,eta+i)/(2**(2*eta))

def quentropy(A,scale, incomplete=True):
    # scaled entropy. This function automatically completes the input set to 1; should this lead to errors due to rounding, use incomplete=False. Is designed to scale with the scaling factor of the binomial coefficient.
    if scale==0:
        return 0
    for i in range(len(A)):
        A[i]=A[i]/scale
    if incomplete:
        A=A+[1-sum(A)]
    return scale*entropy(A,base=2)

In [3]:
def etaSpiked(delta):
    d=delta
    return [d,(1-d)*pbin(3,1)/(1-pbin(3,0)),(1-d)*pbin(3,2)/(1-pbin(3,0)),(1-d)*pbin(3,3)/(1-pbin(3,0)),0]

def uniSearch(eta):
    # creates a uniformly distributed search space
    List=(eta+1)*[1/(2*eta+1)]
    while len(List)<5:
        List+=[0]
    return List

def etaPermSearch(eta,Permaway,lbda=0):
    # creates a search space where all but lbda*pbin many entries from the Permaway-set are replaced with entries from [-eta,eta]\Permaway, accordingly to their respective binomial probability
    w=max(5,eta+1)*[0]
    c=max(5,eta+1)*[0]
    for i in Permaway:
        w[i]=lbda*pbin(eta,i)
        c[i]=pbin(eta,i)
    c=2*sum(c)-c[0]
    for i in range(0,eta+1):
        if not(i in Permaway):
            w[i]=lbda*pbin(eta,i)+(1-lbda)*(comb(2*eta,eta+i,exact=True))/(2**(2*eta)*(1-c))
    return w

def PermSearch(dist,Permaway,lbda=0):
    # creates a search space where all but lbda*p many entries from the Permaway-set are replaced with entries from [-eta,eta]\Permaway, accordingly to their respective distribution
    eta=len(dist)-1
    w=max(5,eta+1)*[0]
    c=max(5,eta+1)*[0]
    for i in Permaway:
        w[i]=lbda*dist[i]
        c[i]=dist[i]
    c=2*sum(c)-c[0]
    for i in range(0,eta+1):
        if not(i in Permaway):
            w[i]=dist[i]*(lbda+(1-lbda)/(1-c))
    return w

def etaSearch(eta):
    # plain old Searchspace where each entry appears pbin(i) many times. Is now called
    return etaPermSearch(eta,[])

def centered_binom_search(eta):
    return etaSearch(eta)

In [4]:
# In general, we use dictionaries for sets of epsilon. In this program, they are referred to as "bibs".

def Nullbib(level):
    # creates a bib of level depth with entry 0 for all epsilon-entries
    out={}
    for i in (10,20,21,22,30,31,32,33):
        for j in range(1,level+1):
            out[i,j]=0
    return out

def levelbib(bib, level):
    #returns all epsilon entries of a specific level
    out={}
    for i in [10,20,21,22,30,31,32,33]:
        out[i]=bib[i,level]
    return out

def copybib(Bib, bibscale=1, epsBib={},scale=1):
    #copies a bib. Can also be used to combine two libraries, i.e. when using the newton search for a new library
    out={}
    for key in Bib:
        if epsBib=={}:
            out[key]=Bib[key]*bibscale
        else:
            out[key]=Bib[key]*bibscale+epsBib[key]*scale
    return out

def formatbib(Bib,level, levelfirst=False, reverselevelfirst=False):
    # takes any input bib and turns it into a bib of the epsilon-format.
    out={}
    if levelfirst:
        for j in range(1,level):
            for i in [10,20,21,22,30,31,32,33]:
                if (i,j) in Bib:
                    out[i,j]=Bib[i,j]
                else:
                    out[i,j]=0
    elif reverselevelfirst:
        for k in range(1,level):
            j=level-k
            for i in [10,20,21,22,30,31,32,33]:
                if (i,j) in Bib:
                    out[i,j]=Bib[i,j]
                else:
                    out[i,j]=0
    else:
        for i in [10,20,21,22,30,31,32,33]:
            for j in range(1,level):
                if (i,j) in Bib:
                    out[i,j]=Bib[i,j]
                else:
                    out[i,j]=0
    return out

def fcrbib(Bib,digits=4,f=round):
    # takes an input bib and rounds its entries
    out={}
    for key in Bib:
        out[key]=f((10**digits)*Bib[key])/(10**digits)
    return out

def idbib(Bib):
    # this function requires no explanation
    return Bib

def countbib(Bib):
    out=0
    for key in Bib:
        if Bib[key]!=0:
            out+=1
    return out

def lightningbib(Bib, scale, gamma=0.001):
    out={}
    for key in Bib:
        if Bib[key]==0:
            out[key]=[0]
        else:
            out[key]=[]
            sign=np.sign(Bib[key])
            for l in range(1,scale+1):
                out[key]+=[sign*l*gamma]
    return out

def normalbib(Bib, gamma=0.001):
    out={}
    for key in Bib:
        out[key]=np.sign(Bib[key])*gamma
    return out

def truthbib(level, tval10=True, tval20=True, tval21=True, tval22=True, tval30=True, tval31=True, tval32=True, tval33=True):
    out={}
    for i in [10,20,21,22,30,31,32,33]:
        val='tval'+str(i)
        for j in range(level):
            out[i,j]=locals()[val]
    return out

def leveltruthbib(tval10=True, tval20=True, tval21=True, tval22=True, tval30=True, tval31=True, tval32=True, tval33=True):
    out={}
    for i in [10,20,21,22,30,31,32,33]:
        out[i,j]=locals()['tval'+str(i)]
    return out

def bintruthbib(level, bstr):
    tval10=str2bool(bstr[0])
    tval20=str2bool(bstr[1])
    tval21=str2bool(bstr[2])
    tval22=str2bool(bstr[3])
    tval30=str2bool(bstr[4])
    tval31=str2bool(bstr[5])
    tval32=str2bool(bstr[6])
    tval33=str2bool(bstr[7])
    return truthbib(level, tval10=tval10, tval20=tval20, tval21=tval21, tval22=tval22, tval30=tval30, tval31=tval31, tval32=tval32, tval33=tval33)

def fullbintruthbib(bstr):
    out={}
    level=floor((len(bstr)-1)/8)
    for j in range(level+1):
        out[10,j+1]=bstr[j*8+0]=='1'
        out[20,j+1]=bstr[j*8+1]=='1'
        out[21,j+1]=bstr[j*8+2]=='1'
        out[22,j+1]=bstr[j*8+3]=='1'
        out[30,j+1]=bstr[j*8+4]=='1'
        out[31,j+1]=bstr[j*8+5]=='1'
        out[32,j+1]=bstr[j*8+6]=='1'
        out[33,j+1]=bstr[j*8+7]=='1'
    return out

In [5]:
def SearchSpace(w):
    # given an input list w, this function returns the entropy of a search space where, for entries from the scale [-eta,eta] (for eta=len(w)+1), an entry i appears relatively w[i] many times
    i=len(w)
    v=[w[0]]
    for i in range(1,i):
        v=v+[w[i]]
        v=v+[w[i]]
    return entropy(v,base=2)

def Representations(w,eps):
    # this function calculates, given a previous level search space list w and an epsilon bib eps, the amount of representations from an entry of upper search spaces with entries from lower levels that are distributed according to the eps bib
    rep0=quentropy([eps[10],eps[10],eps[20],eps[20],eps[30],eps[30]],w[0])
    rep1=quentropy([eps[21],eps[21],eps[31],eps[31],(w[1]-2*eps[21]-2*eps[31])/2,(w[1]-2*eps[21]-2*eps[31])/2],w[1],incomplete=False)
    rep2=quentropy([eps[22],eps[22],eps[32],eps[32]],w[2])
    rep3=quentropy([eps[33],eps[33],(w[3]-2*eps[33])/2,(w[3]-2*eps[33])/2],w[3],incomplete=False)
    return (2*rep3+2*rep2+2*rep1+rep0)

# the cw functions calculate, according to the previous search space and an epsilon bib eps, the search space for the current level
def cw1(pw, ce):
    return (pw[1]+2*pw[2]+pw[3]-2*ce[33]-2*ce[32]-2*ce[31]+2*ce[10]-4*ce[22])/2

def cw2(pw, ce):
    return (pw[3]+2*pw[4]+2*ce[20]+2*ce[21]+2*ce[22]+2*ce[31]-2*ce[33])/2

def cw3(pw, ce):
    return (2*ce[33]+2*ce[32]+2*ce[31]+2*ce[30])/2

def cw(pw, ce):
    c1=cw1(pw, ce)
    c2=cw2(pw, ce)
    c3=cw3(pw, ce)
    c0=1-2*(c1+c2+c3)
    return [c0,c1,c2,c3,0]

In [6]:
#Evaluate_T and Evaluate_L both take an initial distribution as well as an epsilon dictionary bib and calculate each level's respective Time (or space) to create any search list of said level.

def Evaluate(dist, level, bib, retT=True, retL=True):
    T=[0]
    L=[0]
    w=[dist]
    S=[SearchSpace(w[0])]
    R=[1]
    for j in range(1,level):
        lbib=levelbib(bib,j)
        w+=[cw(w[j-1],lbib)]
        T+=[0]
        S+=[SearchSpace(w[j])]
        R+=[Representations(w[j-1],lbib)]
        L+=[S[j]-R[j]]
        if j==level-1:
            R=R+[0]
            L=L+[S[j]/2]
            T=T+[S[j]/2]
            if isnan(R[j]) or isinf(R[j]):
                T[j]=-R[j]
            elif isnan(R[level]) or isinf(R[level]):
                T[j]=R[level]
            else:
                T[j]=2*L[level]-(R[j]-R[level])
        if j>1:
            if isnan(R[j]) or isinf(R[j]):
                T[j-1]=-R[j]
            elif isnan(R[j-1]) or isinf(R[j-1]):
                T[j-1]=-R[j-1]
            else:
                T[j-1]=2*L[j]-(R[j-1]-R[j])
        elif isnan(R[1]) or isinf(R[1]):
            T[0]=-R[1]
        else:
            T[0]=max(0,2*L[1]-(1-R[1]*np.log(2)/np.log(q)))
    if retT and retL:
        return [T,L]
    elif retT:
        return T
    elif retL:
        return L
    return []

def Evaluate_T(dist, level, bib):
    return Evaluate(dist, level, bib, retT=True, retL=False)

def Evaluate_L(dist, level, bib):
    return Evaluate(dist, level, bib, retT=False, retL=True)

def Evaluate_bib(bib):
    for [eta,level] in bib:
        [T,L]=Evaluate(etaPermSearch(eta,range(4,eta+1)),level,bib[eta,level])
        print("------------- eta = " + str(eta) + " , level = " + str(level) + " -------------")
        print("runtime is " + str(max(T)))
        print("memory is " + str(max(L)))
        print()

In [7]:
# Tableprint returns a latex-compilable table of List sizes and run times for a given distribution and a specific epsilon bib. If no specific distribution is entered, this function calculates the distribution according to Permaway and eta

def Tableprint(name, bib, eta=3, Permaway=[], level=4, dist=[], lbda=0, treatzero=" ", ifthrees=True, ifpermute=True, ifcolor=True, iftop=False, ifbottom=False):
    if dist==[]:
        dist=etaSearch(eta)
    pdist=PermSearch(dist,Permaway,lbda=lbda)
    P="{:0.0f}".format(abs(-DistExppermute(dist, Permaway, lbda=lbda)*1000))
    if int(P)<=0:
        P=="0"
    T=Evaluate_T(pdist,level,bib)
    L=Evaluate_L(pdist,level,bib)
    max_T= max(T)
    max_L= max(L)
    for i in range(len(T)):
        if T[i]==max_T:
            T[i]="\mathbf{" + "{:0.0f}".format(ceil(T[i]*1000)) + "}"
        else:
            T[i]="{:0.0f}".format(ceil(T[i]*1000))
        if L[i]==max_L:
            L[i]="\mathbf{" + "{:0.0f}".format(ceil(L[i]*1000)) + "}"
        else:
            L[i]="{:0.0f}".format(ceil(L[i]*1000))
    eggBib={}
    for key in bib:
        if bib[key]>0:
            eggBib[key]="{:0.0f}".format(bib[key]*1000)
        else:
            eggBib[key]=treatzero
    line_segment="\\hhline{~|" + ifstring(ifcolor, ">{\\arrayrulecolor{Gray}}->{\\arrayrulecolor{black}}", nots="~") + "|*{" + ifstring(ifthrees, "8", nots="4") + "}{-}|~|" + ifstring(ifcolor, ">{\\arrayrulecolor{Gray}}->{\\arrayrulecolor{black}}", nots="~") + ifstring(ifpermute,"|~") + "}"
    if iftop:
        printtop(ifthrees=ifthrees, ifpermute=ifpermute, ifcolor=ifcolor)
    print("        \\multirow{"+ str(level+1) + "}*{$" + name + "$}")
    print("        &0&\\multicolumn{" + ifstring(ifthrees, "8", nots="4") + "}{c|}{-}&$" + T[0] + "$&$" + L[0] + "$" + ifstring(ifpermute, "&\\multirow{"+ str(level+1) + "}*{$" + P + "$}") + "\\\\" + line_segment)
    for j in range(1,level-1):
        print("        &" + str(j) + "&$" + eggBib[10,j] + "$&$" + eggBib[20,j] + "$&$" + eggBib[21,j] + "$&$" + eggBib[22,j] + ifstring(ifthrees,"$&$" + eggBib[30,j] + "$&$" + eggBib[31,j] + "$&$" + eggBib[32,j] + "$&$" + eggBib[33,j]) + "$&$" + T[j] + "$&$" + L[j] + "$\\\\")
    print("        &" + str(level-1) + "&$" + eggBib[10,level-1] + "$&$" + eggBib[20,level-1] + "$&$" + eggBib[21,level-1] + "$&$" + eggBib[22,level-1] + ifstring(ifthrees,"$&$" + eggBib[30,level-1] + "$&$" + eggBib[31,level-1] + "$&$" + eggBib[32,level-1] + "$&$" + eggBib[33,level-1]) + "$&$" + T[level-1] + "$&$" + L[level-1] + "$\\\\" + line_segment)
    print("        &" + str(level) + "&\\multicolumn{" + ifstring(ifthrees, "8", nots="4") + "}{c|}{-}&$" + T[level] + "$&$" + L[level] + "$\\\\\\hline")
    if ifbottom:
        printbottom()
        
def printtop(ifthrees=True, ifpermute=True, ifcolor=True):
    print("% IMPORTANT: put \\usepackage{tabular}, \\usepackage{hhline}, \\usepackage{xcolor, colortbl}, \\definecolor{Gray}{gray}{0.9} in your preamble")
    print("    \\begin{tabular}{c|" + ifstring(ifcolor, ">{\\columncolor{Gray}}") + "c|c" + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifthrees, ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c") + ifstring(ifcolor, "c",nots="|c") + ifstring(ifcolor, ">{\\columncolor{Gray}}c",nots="|c")) + "|c|" + ifstring(ifcolor, ">{\\columncolor{Gray}}") + "c" + ifstring(ifpermute, "|c") + "}")
    print("        $\\mathcal D$&$j$&$\\epsilon_{10}^{j}$&$\\epsilon_{20}^{j}$&$\\epsilon_{21}^{j}$&$\\epsilon_{22}^{j}" + ifstring(ifthrees,"$&$\\epsilon_{30}^{j}$&$\\epsilon_{31}^{j}$&$\\epsilon_{32}^{j}$&$\\epsilon_{33}^{j}") + "$&$\\mathcal T^{(j)}$&$\\mathcal L^{(j)}$" + ifstring(ifpermute,"&$p_{good}^{-1}$") + "\\\\\\hline")

def printbottom():
    print("    \\end{tabular}")
    
def ifstring(b,s,nots=""):
    if b:
        return s
    else:
        return nots
    
def resultprint(bib,low=3,promill=0):
    print("    \\[\\begin{tabular}{c|c|c|c|c}")
    print("        $\\eta$&level&$T$&$L$&$P$\\\\\\hline\\hline")
    countbib={}
    levelbib={}
    postdigits=3*promill
    for [eta,level] in bib:
        if eta in countbib:
            countbib[eta]+=1
            levelbib[eta]+=[level]
        else:
            countbib[eta]=1
            levelbib[eta]=[level]
    for eta in countbib:
        if promill==1:
            level=levelbib[eta][0]
            T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            print("        \\multirow{" + str(countbib[eta]) + "}*{$" + str(eta) + "$}&$" + str(level) + "$&$" + str("{:0.3f}".format(ceil(max(T)*1000)/1000)) + "$&$" + str("{:0.3f}".format(ceil(max(L)*1000)/1000)) + "$&\\multirow{" + str(countbib[eta]) + "}*{" + str("{:0.3f}".format(OptExppermute(peta(eta,range(low+1,eta+1)),lbda=0)*1000/1000)) + "}\\\\\cline{2-4}")
            for i in range(1,len(levelbib[eta])-1):
                level=levelbib[eta][i]
                T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
                L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
                print("        &$" + str(level) + "$&$" + str("{:0.3f}".format(ceil(max(T)*1000)/1000)) + "$&$" + str("{:0.3f}".format(ceil(max(L)*1000)/1000)) + "$\\\\\cline{2-4}")
            level=levelbib[eta][len(levelbib[eta])-1]
            T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            print("        &$" + str(level) + "$&$" + str("{:0.3f}".format(ceil(max(T)*1000)/1000)) + "$&$" + str("{:0.3f}".format(ceil(max(L)*1000)/1000)) + "$\\\\\hline")
        else:
            level=levelbib[eta][0]
            T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            print("        \\multirow{" + str(countbib[eta]) + "}*{$" + str(eta) + "$}&$" + str(level) + "$&$" + str("{:0.0f}".format(ceil(max(T)*1000))) + "$&$" + str("{:0.0f}".format(ceil(max(L)*1000))) + "$&\\multirow{" + str(countbib[eta]) + "}*{" + str("{:0.0f}".format(OptExppermute(peta(eta,range(low+1,eta+1)),lbda=0)*1000)) + "}\\\\\cline{2-4}")
            for i in range(1,len(levelbib[eta])-1):
                level=levelbib[eta][i]
                T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
                L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
                print("        &$" + str(level) + "$&$" + str("{:0.0f}".format(ceil(max(T)*1000))) + "$&$" + str("{:0.0f}".format(ceil(max(L)*1000))) + "$\\\\\cline{2-4}")
            level=levelbib[eta][len(levelbib[eta])-1]
            T=Evaluate_T(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            L=Evaluate_L(etaPermSearch(eta,range(low+1,eta+1)),level,bib[eta,level])
            print("        &$" + str(level) + "$&$" + str("{:0.0f}".format(ceil(max(T)*1000))) + "$&$" + str("{:0.0f}".format(ceil(max(L)*1000))) + "$\\\\\hline")
    print("    \\end{tabular}\\]")

In [8]:
def drawbits(l):
    out=""
    for i in range(l):
        out+=str(randint(0,1))
    return out

def str2bool(v):
    return v=="1"

def hamming(bstr):
    out=0
    for i in range(len(bstr)):
        out+=int(bstr[i])
    return out

def create_randset(level, it1=0, it2=100, it3=100, it4=100, it5=0, it6=0, it7=0, it8=0):
    out=[]
    for ham in range(1,9):
        it=locals()['it'+str(ham)]
        for j in range(it):
            randstr=""
            for k in range(level):
                partstr=8*"0"
                while hamming(partstr)!=ham:
                    partstr=drawbits(8)
                randstr+=partstr
            out+=[randstr]
    return out

In [9]:
# unlike with the epsilon libraries, it is trivial to calculate the optimal permutation size, should one require to only permute some but not all Permaway-entries into the irrelevant halt of s||e. OptExppermute calculates the Permutation cost given the Permaway-set and lbda

def TrivExppermute(peta):
    return quentropy([2*peta],1)-2*quentropy([peta],1)

def SmartExppermute(peta,c,lbda=0):
    l=1-lbda
    totalper=quentropy([c],l)+quentropy([c],1)
    goodper=quentropy([c-l*peta],l*(1-peta))+quentropy([c],1-peta)
    return totalper-goodper

def OptExppermute(peta,lbda=0):
    l=1-lbda
    c=l/(1+l)
    return SmartExppermute(peta,c,lbda=lbda)

def UpboundExppermute(a,eta,lbda=0):
    pet=peta(eta,Permaway=range(a,eta+1))
    return OptExppermute(pet,lbda=lbda)

def DistExppermute(dist, Permaway, lbda=0):
    l=1-lbda
    eta=len(dist)-1
    c=(eta+1)*[0]
    for i in Permaway:
        c[i]=l*dist[i]
    peta=2*sum(c)-c[0]
    return quentropy([2*peta],1)-2*quentropy([peta],1)

In [10]:
q=3329

def optimizer(dist, #the only required input asks for the distribution where an optimized bib should be found
              level=4, #asks for the depth of the epsilon bib
              rough=10, #rough trades off initialization time of the eggBib for thoroughness of the starting point
              gamma=0.001, # in the newton round, gamma defines the step size
              delta=0.0001, # delta defines the minimal amount of optimization per newton round that has to be reached before the algorithm terminates
              setupdelta=0.001, # setupdelta says that, given an optimal value optT, what is the minimal improvement that should be made to continue down a given path. trades off runtime for thoroughness
              newtondelta=0, # same as setupdelta for the newton round
              newtonscale=1, # can be used instead of newtondelta for the minimal optimization to scale linearly with the optimal value optT
              Newtonround=True, # if set to false, no newton round takes place. Should generally be left True
              eggBib={}, # if eggBib={}, the program takes a setup step for a decent initialization of the output bib. If a decent epsilon library is known, it can be used as a starting point for the newton round instead
              f=idbib, # f describes the function that modifies the epsilon bib after each newton round. It might be appropriate to set f to fcrbib
              cascadia=False, #if cascadia is True, both setup and newton round only look for epsilon bibs that lead to runtimes where T[j]>=T[j+1] for all j=1,...,level-1. This decreases the runtime of the newton round significantly, but paired with a bad newtondelta can lead to inconvenient premature terminations
              cascadiaerror=0.01, # when paired with cascadia, cascadiaerror allows for T[j+1] to be greater than T[j] by at most cascadiaerror
              lightningiter=1000000, #this defines the upper bound of parameter sets checked when entering the lightning newton round. If the lightning newton round is not wished for, this should be set to 0 instead
              optruthbib={}, # if left empty, every parameter will be optimized. makes runtimes inconceivable when working with >3 levels. In general, it is recommended to only optimize up to four parameters per iteration; this can be achieved by setting optruthbib to either bintruthbib or fullbibtruthbib with randomized inputs (where each level has hamming weight at most 4)
              printset=[] # printset gives the observer the opportunity to see which inputs are currently under observation. If left empty, this does nothing. Does not impact the algorithm itself
             ):
    global setupset
    global newtonset
    optBib=Nullbib(level-1)
    newtonset={}
    if optruthbib=={}:
        optimizebib=truthbib(level)
    else:
        optimizebib=copybib(optruthbib)
    for i in [10,20,21,22,30,31,32,33]:
        for j in range(1,level):
            if optimizebib[i,j]:
                newtonset[i,j]=[-gamma,0,gamma]
            else:
                newtonset[i,j]=[0]
    # throughout this program, eggBib always refers to the epsilon bib that currently is spectated.
    # eggBib either is already predefined, otherwise the first step is to find a starting point. This is a hybrid approach
    if eggBib=={}:
        #initialize Dictionary for epsilon's bib and find the first optimal T-value. Should a library at any point prove to be worse than the current optimal value, it can be discarded immediately. During the course of this program, optT never increases
        eggBib=Nullbib(level-1)
        optT=max(Evaluate_T(dist, level,optBib))
        if rough>0:
            setupset={}
            for i in [10,20,21,22,30,31,32,33]:
                for j in range(1,level):
                    if optimizebib[i,j]:
                        setupset[i,j]=range(rough+1)
                    else:
                        setupset[i,j]=[0]
            # the setup round finds a starting point for the newton round. Should an eggBib be given or no setup Round be asked for (i.e. when rough=0), this step is skipped
            [optT,optBib]=Setup([dist]+level*[0],(level+1)*[0],[SearchSpace(dist)]+level*[0], (level+1)*[0], (level+1)*[0], dist, level, 1, optT, Nullbib(level-1), Nullbib(level-1), rough=rough, setupdelta=setupdelta, cascadia=cascadia, cascadiaerror=cascadiaerror, printset=printset)
    else:
        optBib=formatbib(eggBib,level)
        optT=max(Evaluate_T(dist, level,optBib))
    oldepsBib=Nullbib(level-1)
    while Newtonround==True:
        # the actual optimization part. Every round, the current working bib optBib gets improved until the difference of the old and new runtime is too small
        curT=optT
        curBib=copybib(optBib)
        [optT,optBib,optepsBib]=Newton([dist]+level*[5*[0]], (level+1)*[0], [SearchSpace(dist)]+level*[0], (level+1)*[0], (level+1)*[0], dist,level,1, optT, optBib, optBib, oldepsBib, Nullbib(level-1), Nullbib(level-1), Nullbib(level-1),gamma=gamma, newtondelta=newtondelta, newtonscale=newtonscale,cascadia=cascadia, cascadiaerror=cascadiaerror, lightningiter=lightningiter, printset=printset)
        oldepsBib=copybib(optepsBib)
        optBib=f(optBib)
        optT=max(Evaluate_T(dist,level,optBib))
        if curT-optT<delta:
            Newtonround=False
    return optBib

#The Subroutine for finding a good starting point
def Setup(w, R, S, T, L, dist, toplevel, curlevel, optT, optBib, eggBib, rough=4, setupdelta=0.0001, cascadia=True, cascadiaerror=0.01, printset=[]):
    # Setup goes through all possible epsilons (in steps of size 1/rough) and finds the optimal starting point for the newton round.
    global setupset
    j=curlevel
    prevlevel=j-1
    nextlevel=j+1
    for a10 in setupset[10,j]:
        b10=a10/rough
        if w[prevlevel][0]/2<b10:
            break
        for a20 in setupset[20,j]:
            b20=a20/rough
            if (w[prevlevel][0]-2*b10)/2<b20:
                break
            for a21 in setupset[21,j]:
                b21=a21/rough
                if w[prevlevel][1]/2<b21:
                    break
                for a22 in setupset[22,j]:
                    b22=a22/rough
                    if w[prevlevel][2]/2<b22:
                        break
                    for a30 in setupset[30,j]:
                        b30=a30/rough
                        if (w[prevlevel][0]-2*b10-2*b20)/2<b30:
                            break
                        for a31 in setupset[31,j]:
                            b31=a31/rough
                            if (w[prevlevel][2]-2*b21)/2<2*b31:
                                break
                            for a32 in setupset[32,j]:
                                b32=a32/rough
                                if (w[prevlevel][2]-2*b22)/2<b32:
                                    break
                                for a33 in setupset[33,j]:
                                    b33=a33/rough
                                    if w[prevlevel][3]<2*b33:
                                        break
                                    eggBib[10,j]=b10
                                    eggBib[20,j]=b20
                                    eggBib[21,j]=b21
                                    eggBib[22,j]=b22
                                    eggBib[30,j]=b30
                                    eggBib[31,j]=b31
                                    eggBib[32,j]=b32
                                    eggBib[33,j]=b33
                                    lbib=levelbib(eggBib,j)
                                    w[j]=cw(w[prevlevel],lbib)
                                    S[j]=SearchSpace(w[j])
                                    R[j]=Representations(w[prevlevel],lbib)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    for k in range(j,toplevel+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==toplevel-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(eggBib)
                                            print(T)
                                            print(newT)
                                            optT=newT
                                            optBib=copybib(eggBib)
                                    elif newT<optT-setupdelta:
                                        if not(cascadia) or j<=2:
                                            [optT,optBib]=Setup(w, R, S, T, L, dist, toplevel, nextlevel, optT,optBib,eggBib, rough=rough,cascadia=cascadia, cascadiaerror=cascadiaerror, printset=printset)
                                        elif T[prevlevel]<T[prevlevel-1]+cascadiaerror:
                                            [optT,optBib]=Setup(w, R, S, T, L, dist, toplevel, nextlevel, optT,optBib,eggBib, rough=rough,cascadia=cascadia, cascadiaerror=cascadiaerror, printset=printset)
    return [optT,optBib]

def Newton(w, R, S, T, L, dist,toplevel,curlevel,optT, optBib, oldBib, oldepsBib, eggBib, epsBib, optepsBib,gamma=0.001, newtondelta=0.0001, newtonscale=0.99, cascadia=True, cascadiaerror=0.01, lightningiter=100000, printset=[]):
    global newtonset
    j=curlevel
    prevlevel=j-1
    nextlevel=j+1
    for eps10 in newtonset[10,j]:
        if 0>oldBib[10,j]+eps10 or w[prevlevel][0]/2<oldBib[10,j]+eps10 or eps10*oldepsBib[10,j]<0:
            continue
        for eps20 in newtonset[20,j]:
            if 0>oldBib[20,j]+eps20 or (w[prevlevel][0]-2*(oldBib[10,j]+eps10))/2<oldBib[20,j]+eps20 or eps20*oldepsBib[20,j]<0:
                continue
            for eps21 in newtonset[21,j]:
                if 0>oldBib[21,j]+eps21 or w[prevlevel][1]/2<oldBib[21,j]+eps21 or eps21*oldepsBib[21,j]<0:
                    continue
                for eps22 in newtonset[22,j]:
                    if 0>oldBib[22,j]+eps22 or w[prevlevel][2]/2<oldBib[22,j]+eps22 or eps22*oldepsBib[22,j]<0:
                        continue
                    for eps32 in newtonset[32,j]:
                        if 0>oldBib[32,j]+eps32 or (w[prevlevel][2]/2-2*(oldBib[22,j]+eps22))/2<oldBib[32,j]+eps32 or eps32*oldepsBib[32,j]<0:
                            continue
                        for eps33 in newtonset[33,j]:
                            if 0>oldBib[33,j]+eps33 or w[prevlevel][3]/2<oldBib[33,j]+eps33 or eps33*oldepsBib[33,j]<0:
                                continue
                            for eps31 in newtonset[31,j]:
                                if 0>oldBib[31,j]+eps31 or (w[prevlevel][1]-2*(oldBib[21,j]+eps21))/2<oldBib[31,j]+eps31 or eps31*oldepsBib[31,j]<0:
                                    continue
                                for eps30 in newtonset[30,j]:
                                    if 0>oldBib[30,j]+eps30 or (w[prevlevel][0]-2*(oldBib[20,j]+eps20+oldBib[10,j]+eps10))/2<oldBib[30,j]+eps30 or eps30*oldepsBib[30,j]<0:
                                        continue
                                    eggBib[10,j]=oldBib[10,j]+eps10
                                    eggBib[20,j]=oldBib[20,j]+eps20
                                    eggBib[21,j]=oldBib[21,j]+eps21
                                    eggBib[22,j]=oldBib[22,j]+eps22
                                    eggBib[30,j]=oldBib[30,j]+eps30
                                    eggBib[31,j]=oldBib[31,j]+eps31
                                    eggBib[32,j]=oldBib[32,j]+eps32
                                    eggBib[33,j]=oldBib[33,j]+eps33
                                    epsBib[10,j]=eps10
                                    epsBib[20,j]=eps20
                                    epsBib[21,j]=eps21
                                    epsBib[22,j]=eps22
                                    epsBib[30,j]=eps30
                                    epsBib[31,j]=eps31
                                    epsBib[32,j]=eps32
                                    epsBib[33,j]=eps33
                                    lbib=levelbib(eggBib,j)
                                    w[j]=cw(w[j-1],lbib)
                                    S[j]=SearchSpace(w[j])
                                    R[j]=Representations(w[prevlevel],lbib)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    T[j]=0
                                    for k in range(j+1,toplevel+1):
                                        T[k]=0
                                        R[k]=0
                                        L[k]=0
                                        S[k]=0
                                        w[k]=5*[0]
                                    newT=max(T)
                                    if j==toplevel-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(eggBib)
                                            print(T)
                                            print(newT)
                                            optT=newT
                                            optBib=copybib(eggBib)
                                            optepsBib=copybib(epsBib)
                                            notnull=countbib(epsBib)
                                            if notnull>0:
                                                scale=floor(lightningiter**(1/notnull))
                                                if scale>1:
                                                    Lightningsets=lightningbib(epsBib,scale,gamma=gamma)
                                                    [optT,optBib,optepsBib]=Lightningnewton([dist]+toplevel*[5*[0]], (toplevel+1)*[0], [SearchSpace(dist)]+toplevel*[0], (toplevel+1)*[0], (toplevel+1)*[0], dist,toplevel,1, optT, optBib, oldBib, Nullbib(toplevel-1), Nullbib(toplevel-1), optepsBib, Lightningsets, gamma=gamma, cascadia=cascadia, cascadiaerror=cascadiaerror, printset=printset)
                                                else:
                                                    clear_output(wait=True)
                                                    for entry in printset:
                                                        print(entry)
                                                    print(eggBib)
                                                    print(T)
                                                    print(newT)
                                                    optT=newT
                                                    optBib=copybib(eggBib)
                                                    optepsBib=copybib(epsBib)
                                    elif newT<newtonscale*optT-newtondelta:
                                        if not(cascadia) or j<=2:
                                            [optT,optBib,optepsBib]=Newton(w, R, S, T, L, dist,toplevel,nextlevel,optT, optBib, oldBib, oldepsBib, eggBib, epsBib, optepsBib,gamma=gamma, newtondelta=newtondelta, newtonscale=newtonscale, cascadia=cascadia, cascadiaerror=cascadiaerror, lightningiter=lightningiter, printset=printset)
                                        elif T[prevlevel]<T[prevlevel-1]+cascadiaerror:
                                            [optT,optBib,optepsBib]=Newton(w, R, S, T, L, dist,toplevel,nextlevel,optT, optBib, oldBib, oldepsBib, eggBib, epsBib, optepsBib,gamma=gamma, newtondelta=newtondelta, newtonscale=newtonscale, cascadia=cascadia, cascadiaerror=cascadiaerror, lightningiter=lightningiter, printset=printset)
    return [optT,optBib,optepsBib]

def Lightningnewton(w, R, S, T, L, dist, toplevel,curlevel,optT, optBib, oldBib, eggBib, epsBib, optepsBib, Lightningsets, gamma=0.001, cascadia=True, cascadiaerror=0.01, printset=[]):
    j=curlevel
    prevlevel=j-1
    nextlevel=j+1
    for eps10 in Lightningsets[10,j]:
        if 0>oldBib[10,j]+eps10 or w[prevlevel][0]/2<oldBib[10,j]+eps10:
            continue
        for eps20 in Lightningsets[20,j]:
            if 0>oldBib[20,j]+eps20 or (w[prevlevel][0]-2*(oldBib[10,j]+eps10))/2<oldBib[20,j]+eps20:
                continue
            for eps21 in Lightningsets[21,j]:
                if 0>oldBib[21,j]+eps21 or w[prevlevel][1]/2<oldBib[21,j]+eps21:
                    continue
                for eps22 in Lightningsets[22,j]:
                    if 0>oldBib[22,j]+eps22 or w[prevlevel][2]/2<oldBib[22,j]+eps22:
                        continue
                    for eps32 in Lightningsets[32,j]:
                        if 0>oldBib[32,j]+eps32 or (w[prevlevel][2]/2-2*(oldBib[22,j]+eps22))/2<oldBib[32,j]+eps32:
                            continue
                        for eps33 in Lightningsets[33,j]:
                            if 0>oldBib[33,j]+eps33 or w[prevlevel][3]/2<oldBib[33,j]+eps33:
                                continue
                            for eps31 in Lightningsets[31,j]:
                                if 0>oldBib[31,j]+eps31 or (w[prevlevel][1]-2*(oldBib[21,j]+eps21))/2<oldBib[31,j]+eps31:
                                    continue
                                for eps30 in Lightningsets[30,j]:
                                    if 0>oldBib[30,j]+eps30 or (w[prevlevel][0]-2*(oldBib[20,j]+eps20+oldBib[10,j]+eps10))/2<oldBib[30,j]+eps30:
                                        continue
                                    eggBib[10,j]=oldBib[10,j]+eps10
                                    eggBib[20,j]=oldBib[20,j]+eps20
                                    eggBib[21,j]=oldBib[21,j]+eps21
                                    eggBib[22,j]=oldBib[22,j]+eps22
                                    eggBib[30,j]=oldBib[30,j]+eps30
                                    eggBib[31,j]=oldBib[31,j]+eps31
                                    eggBib[32,j]=oldBib[32,j]+eps32
                                    eggBib[33,j]=oldBib[33,j]+eps33
                                    epsBib[10,j]=eps10
                                    epsBib[20,j]=eps20
                                    epsBib[21,j]=eps21
                                    epsBib[22,j]=eps22
                                    epsBib[30,j]=eps30
                                    epsBib[31,j]=eps31
                                    epsBib[32,j]=eps32
                                    epsBib[33,j]=eps33
                                    lbib=levelbib(eggBib,j)
                                    w[j]=cw(w[j-1],lbib)
                                    S[j]=SearchSpace(w[j])
                                    R[j]=Representations(w[prevlevel],lbib)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-(1-R[1]*np.log(2)/np.log(q)))
                                    for k in range(j,toplevel+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==toplevel-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(eggBib)
                                            print(T)
                                            print(newT)
                                            optT=newT
                                            optBib=copybib(eggBib)
                                            optepsBib=normalbib(epsBib, gamma=gamma)
                                    elif newT<optT:
                                        if cascadia and j>2 and T[prevlevel]<T[prevlevel-1]+cascadiaerror:
                                            [optT,optBib,optepsBib]=Lightningnewton(w, R, S, T, L, dist, toplevel, nextlevel, optT, optBib, oldBib, eggBib, epsBib, optepsBib, Lightningsets, gamma=gamma, printset=printset)
                                        elif not(cascadia) or j<=2:
                                            [optT,optBib,optepsBib]=Lightningnewton(w, R, S, T, L, dist, toplevel, nextlevel, optT, optBib, oldBib, eggBib, epsBib, optepsBib, Lightningsets, gamma=gamma, printset=printset)
    return [optT,optBib,optepsBib]

In [15]:
# this is the optimization part of the code. For all possible levels and etas, it creates a bib and optimizes it by randomly drawing level parameters which it alters in hopes of finding optimization. It is recommended to find a good starting point by setting the first optruthbib to bintruthbib('11110000').
optibib={}
randset=[8*'10000000']
for i in range(3,4):
    optibib[i,1]={}
    for level in range(2,5):
        optibib[i,level]={}
        for randbits in randset:
            optibib[i,level]=optimizer(etaSearch(3),optruthbib=fullbintruthbib(randbits),newtondelta=0, eggBib=optibib[i,level], level=level,f=fcrbib, printset=[level, randbits])

4
1000000010000000100000001000000010000000100000001000000010000000
{(10, 1): 0.073, (10, 2): 0.028, (10, 3): 0.0, (20, 1): 0.0, (20, 2): 0.0, (20, 3): 0.0, (21, 1): 0.0, (21, 2): 0.0, (21, 3): 0.0, (22, 1): 0.0, (22, 2): 0.0, (22, 3): 0.0, (30, 1): 0.0, (30, 2): 0.0, (30, 3): 0.0, (31, 1): 0.0, (31, 2): 0.0, (31, 3): 0.0, (32, 1): 0.0, (32, 2): 0.0, (32, 3): 0.0, (33, 1): 0.0, (33, 2): 0.0, (33, 3): 0.0}
[0.5168409802129491, 0.7867332291899143, 0.49412387338544517, 0.5020672830681523, 0.43272114153407615]
0.7867332291899143


In [118]:
ifthrees=True
ifpermute=True
printtop(ifthrees=ifthrees, ifpermute=ifpermute, ifcolor=True)
for key in [6,10,12]:
    for level in (2,7):
        bib=optiFrodo[key,level]
        name=Frodo_Name[key]
        dist=Frodo_Dist[key]
        Permaway=range(4,key+1)
        Tableprint(name, bib, level=level, dist=dist, Permaway=Permaway, ifcolor=True, ifthrees=ifthrees, ifpermute=ifpermute)
printbottom()

% IMPORTANT: put \usepackage{tabular}, \usepackage{hhline}, \usepackage{xcolor, colortbl}, \definecolor{Gray}{gray}{0.9} in your preamble
    \begin{tabular}{c|>{\columncolor{Gray}}c|c>{\columncolor{Gray}}cc>{\columncolor{Gray}}cc>{\columncolor{Gray}}cc>{\columncolor{Gray}}c|c|>{\columncolor{Gray}}c|c}
        $\mathcal D$&$j$&$\epsilon_{10}^{j}$&$\epsilon_{20}^{j}$&$\epsilon_{21}^{j}$&$\epsilon_{22}^{j}$&$\epsilon_{30}^{j}$&$\epsilon_{31}^{j}$&$\epsilon_{32}^{j}$&$\epsilon_{33}^{j}$&$\mathcal T^{(j)}$&$\mathcal L^{(j)}$&$p_{good}^{-1}$\\\hline
        \multirow{3}*{$\chi_{\text{Frodo-1344}}$}
        &0&\multicolumn{8}{c|}{-}&$817$&$0$&\multirow{3}*{$25$}\\\hhline{~|>{\arrayrulecolor{Gray}}->{\arrayrulecolor{black}}|*{8}{-}|~|>{\arrayrulecolor{Gray}}->{\arrayrulecolor{black}}|~}
        &1&$131$&$ $&$ $&$ $&$ $&$ $&$ $&$ $&$871$&$871$\\\hhline{~|>{\arrayrulecolor{Gray}}->{\arrayrulecolor{black}}|*{8}{-}|~|>{\arrayrulecolor{Gray}}->{\arrayrulecolor{black}}|~}
        &2&\multicolumn{8}

In [104]:
# this is the optimization part of the code. For all possible levels and etas, it creates a bib and optimizes it by randomly drawing level parameters which it alters in hopes of finding optimization. It is recommended to find a good starting point by setting the first optruthbib to bintruthbib('11110000').
optiFrodo={}
randset=[8*'11110000']+create_randset(8)
for key in [6,10,12]:
    optiFrodo[key,1]={}
    for level in range(2,9):
        optiFrodo[key,level]=optiFrodo[key,level-1]
        for randbits in randset:
            optiFrodo[key,level]=optimizer(PermSearch(Frodo_Dist[key],range(4,key+1)),optruthbib=fullbintruthbib(randbits),newtondelta=0, eggBib=optiFrodo[key,level], level=level,f=fcrbib, printset=[key, randbits])

12
0110110010010101100011011101000110010101101100101010101010111000
{(10, 1): 0.04, (10, 2): 0.056, (10, 3): 0.056, (10, 4): 0.042, (10, 5): 0.015, (10, 6): 0.002, (20, 1): 0.019, (20, 2): 0.008, (20, 3): 0.002, (20, 4): 0.0, (20, 5): 0.0, (20, 6): 0.0, (21, 1): 0.029, (21, 2): 0.025, (21, 3): 0.011, (21, 4): 0.002, (21, 5): 0.0, (21, 6): 0.0, (22, 1): 0.023, (22, 2): 0.025, (22, 3): 0.016, (22, 4): 0.003, (22, 5): 0.0, (22, 6): 0.0, (30, 1): 0.007, (30, 2): 0.0, (30, 3): 0.0, (30, 4): 0.0, (30, 5): 0.0, (30, 6): 0.0, (31, 1): 0.011, (31, 2): 0.001, (31, 3): 0.0, (31, 4): 0.0, (31, 5): 0.0, (31, 6): 0.0, (32, 1): 0.011, (32, 2): 0.003, (32, 3): 0.0, (32, 4): 0.0, (32, 5): 0.0, (32, 6): 0.0, (33, 1): 0.02, (33, 2): 0.004, (33, 3): 0.0, (33, 4): 0.0, (33, 5): 0.0, (33, 6): 0.0}
[0, 0.4335720997714694, 0.4336205399141839, 0.43261326671517675, 0.4332944713172906, 0.4329528271135391, 0.3705340712170829, 0.31782381830397777]
0.4336205399141839
